In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Oct_dict_JSON.json', 'r') as json_file:
    link_day_minute_Oct_dict_JSON = json.load(json_file)

In [10]:
week_day_Oct_list = [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]

In [11]:
link_day_minute_Oct_list = []
for link_idx in range(24):
    for day in week_day_Oct_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Oct_list.append(link_day_minute_Oct_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Oct_list)

66240

In [13]:
x = np.matrix(link_day_minute_Oct_list)
x = np.matrix.reshape(x, 24, 2760)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2760)

In [15]:
x[:,:2]

matrix([[ 1228.52259938,  1227.58379738],
        [ 1205.70143602,  1198.19920079],
        [ 1472.80842838,  1495.06661697],
        [ 1525.20789748,  1551.71420734],
        [ 1549.70396382,  1514.49201108],
        [ 1352.76189511,  1365.06458183],
        [ 1268.67487868,  1279.66882338],
        [ 1267.1214019 ,  1256.19370661],
        [ 1653.11834786,  1527.857326  ],
        [ 1692.64550575,  1688.56117386],
        [ 1555.41146663,  1543.07128769],
        [ 1225.47605676,  1303.35358944],
        [ 1178.26581789,  1146.39568106],
        [ 1350.6981329 ,  1360.15396894],
        [ 1266.99741002,  1298.97358733],
        [  931.07432394,   902.82359552],
        [ 1254.74934375,  1252.12090991],
        [ 1235.65698659,  1247.53684481],
        [ 1534.16177308,  1526.19551456],
        [ 1589.82194229,  1566.86546908],
        [ 1365.07968644,  1372.30851882],
        [ 1310.36426964,  1360.30826713],
        [ 1179.29797694,  1216.19718826],
        [ 1279.18800817,  1225.697

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+01, 1e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.27347853e+07  0.00000000e+00  7.86e+03 8.75e+02  1.00e+06     0s
   1  -5.49351967e+06 -5.67689324e+06  1.15e+03 1.44e+02  2.25e+05     0s
   2  -3.03454608e+06 -4.56379911e+06  2.46e+01 2.88e+00  3.09e+04     0s
   3  -3.32312744e+06 -3.54435692e+06  1.13e-01 1.26e-02  3.9

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Oct_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[7.245262672833899e-12,
 1450.2787264487033,
 7.952910768343458e-12,
 0.000431070877524007,
 1183.2445118569408,
 0.0007236434020286356,
 2.399934342918424e-09,
 0.00012371883115890915,
 0.0037890006992570835,
 1225.1498044403988,
 187.76054880256095,
 0.000247683626582066,
 0.0006101576215438755,
 0.0006867313363166233,
 1868.96101179716,
 950.2385016638945,
 0.6545315768152449,
 269.7212873392095,
 0.0002941751710005599,
 0.0007239903509816195,
 0.0007136216564262166,
 751.8582628447087,
 576.8364934240168,
 0.043055284564384344,
 371.0661818142231,
 676.6303521741377,
 0.006020887924381102,
 1210.7409067460105,
 0.03216188318988622,
 0.0258462981995613,
 893.948181529033,
 1171.7684990628734,
 0.03534323839803244,
 256.02338158926085,
 1228.906773696326,
 1.304555323945194e-06,
 1.3045536138847288e-06,
 1.3045677922938538e-06,
 0.05246974679731542,
 394.27942969257174,
 846.6561029305315,
 0.029674032471216027,
 0.004895764317762341,
 0.004869614576949122,
 0.005309171261589307,
 0.